Hello people,
In this Notebook, We use numpy array and CNN to classify freshly fish from rotten ones.The benefit of using numpy array will give you more chance to apply direffent technique in this dataset. Dont forget to UPVOTE  If you feel this notebook will help you to do better in emotion recognition.

Also You can see my existing paper on Emotion Recognition on FER13 Dataset :[Facial Emotion Recognition by Ensemble-DenseNet Networks](https://ieeexplore.ieee.org/document/10461880/authors#authors)

You can Download the paper from here: [ResearchGate](https://www.researchgate.net/publication/379106612_Facial_Emotion_Recognition_by_Ensemble-DenseNet_Networks)

In [2]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt

In [3]:
import os

def find_Class(directory_path):
    if not os.path.exists(directory_path):
        raise ValueError(f"The directory '{directory_path}' does not exist.")
    if not os.path.isdir(directory_path):
        raise ValueError(f"The path '{directory_path}' is not a directory.")
    
    # Get a list of all entries in the directory
    all_entries = os.listdir(directory_path)
    
    # Filter out only directories
    folders = [entry for entry in all_entries if os.path.isdir(os.path.join(directory_path, entry))]
    
    return folders

# Find all The classes 

In [4]:
DIRECTORY= r"/kaggle/input/fish-freshness-classification/Resized"
CATAGORIES= []
try:
    folders = find_Class(DIRECTORY)
    print(f"Directories in '{DIRECTORY}':")
    for folder in folders:
        CATAGORIES.append(folder)
except ValueError as e:
    print(e)

CATAGORIES

Directories in '/kaggle/input/fish-freshness-classification/Resized':


['Fish_A', 'Fish_B']

# Convert data to numpy array

In [12]:
data=[]
img_size=224

for categories in CATAGORIES:
    folder=os.path.join(DIRECTORY,categories)
    label=CATAGORIES.index(categories)
    
    
    for img in os.listdir(folder):
        img=os.path.join(folder,img)
        img_arr=cv2.imread(img)
        if img_arr is not None:  # Check if the image is successfully loaded
            img_arr = cv2.resize(img_arr, (img_size, img_size))
            data.append([img_arr, label])
        else:
            print(f"Failed to load image {img}")
            
#         img_arr=cv2.resize(img_arr,(100,100))
        
#         data.append([img_arr,label])
        

In [13]:
len(data)

4476

# mergin test data into train data

In [8]:
# data

# Shuffle data for better outcome

In [14]:
random.shuffle(data)

# Separating level and image  array data

In [15]:
x=[]
y=[]


for features,label in data:
    x.append(features)
    y.append(label)


In [16]:
X= np.array(x)
Y=np.array(y)

In [12]:
# x

# Normalize the image array between 0-1

In [17]:
X=X/255
# X

In [18]:
X.shape

(4476, 224, 224, 3)

In [19]:
Y.shape

(4476,)

# Create Model

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

2024-05-26 21:38:03.289174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 21:38:03.289269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 21:38:03.414145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [22]:
model=Sequential()
model.add( Conv2D(64,(3,3),input_shape=X.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add( Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add( Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(2,activation='softmax'))

In [23]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpoint=ModelCheckpoint(r'fer.keras',
                          monitor='val_accuracy',
                          mode='max',
                          save_best_only=True,
                          verbose=1)
earlystop=EarlyStopping(monitor='val_accuracy',
                        mode='max',
                       min_delta=0.001,
                       patience=20,
                       verbose=1,
                       restore_best_weights=True)

callbacks=[checkpoint,earlystop]

# Train model

In [24]:
model.fit(X,Y,epochs=50,validation_split=0.2,callbacks =callbacks)

Epoch 1/50
  5/112 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.5114 - loss: 1.0926

I0000 00:00:1716759534.369051     134 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.7554 - loss: 0.5268
Epoch 1: val_accuracy improved from -inf to 0.97321, saving model to fer.keras
112/112 ━━━━━━━━━━━━━━━━━━━━ 18s 92ms/step - accuracy: 0.7564 - loss: 0.5248 - val_accuracy: 0.9732 - val_loss: 0.0910
Epoch 2/50
111/112 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9458 - loss: 0.1924
Epoch 2: val_accuracy improved from 0.97321 to 0.97433, saving model to fer.keras
112/112 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.9459 - loss: 0.1926 - val_accuracy: 0.9743 - val_loss: 0.0958
Epoch 3/50
111/112 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9523 - loss: 0.2596
Epoch 3: val_accuracy improved from 0.97433 to 0.97656, saving model to fer.keras
112/112 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.9525 - loss: 0.2595 - val_accuracy: 0.9766 - val_loss: 0.2365
Epoch 4/50
111/112 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9672 - loss: 0.3995
Epoch 4: val_accuracy did not improve from 0.97656
112/112 ━━━━━━

In [25]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │        43,266 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 301,352 (1.15 MB)

 Trainable params: 100,450 (392.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 200,902 (784.78 KB)

# Prediction Using Model

In [37]:
from keras.preprocessing import image
import numpy as np

img_pred=image.load_img(r"/kaggle/input/fish-freshness-classification/Resized/Fish_A/DSC06219.JPG",target_size=(img_size,img_size))

img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred, axis=0)


rslt= model.predict(img_pred)
print ("Original Class: Fish_A")
print(rslt)
print(f"Predicted Class: {CATAGORIES[np.argmax(rslt)]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Original Class: Fish_A
[[1. 0.]]
Predicted Class: Fish_A
